In [1]:
import os
from os import walk

def find_files_recursively(folder, fname_filter):
    files = []
    for root, directories, filenames in os.walk(folder):       
        for filename in filenames:
            if filename == fname_filter:
                files.append((root, filename))
    return sorted(files)

In [9]:
import pickle
from Settings import Settings
from FindFiles import find_files

DATASET = "CoralBleaching"
#DATASET = "SkinCancer"

EMPTY_TAG = "Empty"
ANAPHORA = "Anaphor"

settings = Settings()
training_pickled = settings.data_directory + DATASET + "/Thesis_Dataset/training.pl"
training_data_folder = settings.data_directory + DATASET + "/Thesis_Dataset/Training"
# root of all files for the corpus dataset
root_essay_dir = settings.data_directory + DATASET + "/"

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [10]:
# do I need to do this? good for validation below, but not needed otherwise
with open(training_pickled, "rb+") as f:
    untagged_essays_train = pickle.load(f)

len(untagged_essays_train)# 902

902

In [11]:
from collections import defaultdict
# list some essays with ana tags
name2essay = {}
ana_tag_locations = []

essay2ana_sent_ixs = defaultdict(set)

for e in untagged_essays_train:
    name2essay[e.name] = e
    # heuristic for usefulness of essay - number of distinct tags
    unique_tags = set()    
    for sentix, sent in enumerate(e.sentences):
        for wdix, (wd, tags) in enumerate(sent):
            unique_tags.update(tags)
    
    for sentix, sent in enumerate(e.sentences):
        for wdix, (wd, tags) in enumerate(sent):
            if ANAPHORA in tags:
                ana_tag_locations.append((e.name, sentix, wdix, len(unique_tags)))
                essay2ana_sent_ixs[e.name].add(sentix)

In [12]:
srted_ana_tag_locations = sorted(ana_tag_locations, key = lambda tpl: -tpl[3])[:20]
list(enumerate(srted_ana_tag_locations))

[(0, ('EBA1415_LZBA_4_CB_ES-05530.ann', 0, 15, 49)),
 (1, ('EBA1415_LZBA_4_CB_ES-05530.ann', 0, 16, 49)),
 (2, ('EBA1415_LZBA_4_CB_ES-05530.ann', 6, 1, 49)),
 (3, ('EBA1415_BLRW_3_CB_ES-05173.ann', 6, 0, 48)),
 (4, ('EBA1415_LZBA_3_CB_ES-05505.ann', 5, 0, 45)),
 (5, ('EBA1415_RCGJ_4a_CB_ES-04684.ann', 7, 0, 43)),
 (6, ('EBA1415_SERS_1314_CB_ES-05098.ann', 7, 0, 43)),
 (7, ('EBA1415_KYNS_4_CB_ES-05390.ann', 10, 1, 42)),
 (8, ('EBA1415_LZBA_4_CB_ES-05531.ann', 3, 0, 41)),
 (9, ('EBA1415_LZBA_4_CB_ES-05531.ann', 12, 0, 41)),
 (10, ('EBA1415_LZBA_4_CB_ES-05531.ann', 12, 1, 41)),
 (11, ('EBA1415_SDMK_4_CB_ES-04758.ann', 5, 0, 41)),
 (12, ('EBA1415_SDMK_6_CB_ES-04773.ann', 1, 0, 41)),
 (13, ('EBA1415_SDMK_6_CB_ES-04773.ann', 5, 19, 41)),
 (14, ('EBA1415_SDMK_6_CB_ES-04773.ann', 12, 0, 41)),
 (15, ('EBA1415_TFHC_1_CB_ES-05941.ann', 3, 5, 41)),
 (16, ('EBA1415_TFHC_1_CB_ES-05941.ann', 6, 0, 41)),
 (17, ('EBA1415_TFHC_1_CB_ES-05941.ann', 8, 5, 41)),
 (18, ('EBA1415_TFHC_1_CB_ES-05941.ann', 14, 

In [26]:
PREFIX_LEN = 7
ix = 0

# essay_name, _, _, _ = srted_ana_tag_locations[ix]
essay_name = 'EBA1415_TRJB_2_CB_ES-05021.ann'
print(essay_name)
print()

e = name2essay[essay_name]
        
sents = essay2ana_sent_ixs[essay_name]

for sentix in sents:
    sent = e.sentences[sentix]
    
    max_wd_len = 0
    ana_tag_ixs = set()
    for wdix, (wd,tags) in list(enumerate(sent)):
        max_wd_len = max(len(wd), max_wd_len)
        if ANAPHORA in tags:
            ana_tag_ixs.add(wdix)
            print(wdix,wd,tags)

    # print sentence
    print("Sentence No:", sentix+1)
    for wdix, (wd,tags) in list(enumerate(sent)):
        prefix = " " * PREFIX_LEN
        if wdix in ana_tag_ixs:
            prefix = "*" * PREFIX_LEN
        print(prefix + " " + wd.ljust(max_wd_len+1) + str(sorted(tags, key = lambda tag: (len(tag),tag))))
    print("-" * 80)
    
print()
# list all locations that this file is found (for locating in Bratt)

files = find_files_recursively(root_essay_dir, essay_name)
print("File:")
print(essay_name)
print()
print("Directories:")
for f in list(zip(*files))[0]:
    print(f)

EBA1415_TRJB_2_CB_ES-05021.ann

2 this {'Anaphor'}
Sentence No: 17
        in         []
        conclusion []
******* this       ['Anaphor']
        is         []
        why        []
        they       ['50']
        turn       ['50']
        white      ['50', 'COMPILED']
        .          []
--------------------------------------------------------------------------------
0 under {'Anaphor', 'Causer', 'Causer:Anaphor', 'Causer:Anaphor->Result:2'}
1 these {'Anaphor', 'Causer', 'Causer:Anaphor', 'Causer:Anaphor->Result:2'}
2 circumstances {'Anaphor', 'Causer', 'Causer:Anaphor', 'Causer:Anaphor->Result:2'}
Sentence No: 13
******* under         ['Causer', 'Anaphor', 'Causer:Anaphor', 'Causer:Anaphor->Result:2']
******* these         ['Causer', 'Anaphor', 'Causer:Anaphor', 'Causer:Anaphor->Result:2']
******* circumstances ['Causer', 'Anaphor', 'Causer:Anaphor', 'Causer:Anaphor->Result:2']
        ,             ['explicit']
        warm          ['2', 'Result', 'Result:2', 'Causer:Anapho

In [27]:
print(essay_name)
full_path = os.path.join(training_data_folder, essay_name)
with open(full_path) as f:
    lines = f.readlines()

references = set()
for line in lines:
    if ANAPHORA in line:
        tag = line.split("\t")[0].strip()
        references.add(tag)
        
print("Anaphora tags:" + str(references))
print()

print("Contents")
for line in lines:
    prefix = " " * PREFIX_LEN     
    if ANAPHORA in line:
        prefix = "*" * PREFIX_LEN
    else:
        for tag in references:
            if tag + " " in line or line.strip().endswith(tag):
                 prefix = ("R: " + tag).ljust(PREFIX_LEN)
    print(prefix + " " + line.strip())

EBA1415_TRJB_2_CB_ES-05021.ann
Anaphora tags:{'T29', 'T26', 'T30', 'T34'}

Contents
        T1	50 29 44	coral bleaching
        T2	11 81 90	bad storm
        T3	6 102 125	stressors the coral out
        T4	50 133 148	loses its color
        A1	Vague T4
        T5	explicit 76 78;99 101	If it
        E1	explicit:T5 Causer:T2 Result:T3
        T6	explicit 130 132	it
        E2	explicit:T6 Causer:T3 Result:T4
        T7	50 229 264	algae helps the coral get its color
        A2	Normal T7
        T8	6 273 292	coral is stress out
        T9	7 302 327	coral won't eat the algae
        A3	Vague T9
        T10	50 336 393	coral won't get its color for which is why it turns white
        T11	explicit 266 268;293 297	If then
        E3	explicit:T11 Causer:T8 Result:T9
        T12	explicit 328 331	and
        E4	explicit:T12 Causer:T9 Result:T10
        T13	3 454 472	water temperatures
        A4	Vague T13
        T14	1 477 488	trade winds
        A5	Vague T14
        T15	3 499 524	water temperature

## Determine How the Relation Codes Are Defined

In [15]:
all_files = sorted(find_files(training_data_folder, ".*.ann"))
len(all_files)

902

In [16]:
rel_ships = defaultdict(int)
for f in all_files:
    with open(f) as fin:
        lines = fin.readlines()
    for l in lines:
        l = l.strip()
        if not l:
            continue
        if l[0] == "R":
            rel_ships[l]+=1
len(rel_ships)            

229

In [ ]:
l_srtd = sorted(rel_ships.items(), key = lambda tpl: -tpl[1])
l_srtd[0:10]

In [ ]:
for l, freq in l_srtd:
    assert "antecedent" in l, l
    tokens = l.replace("\t"," ").split()
    assert len(tokens) == 4, len(tokens)
    id, ante, arg1, arg2 = tokens
    assert arg1.startswith("Arg1")
    assert arg2.startswith("Arg2")
    arg1_code = arg1.split(":")[1]
    arg2_code = arg2.split(":")[1]
    print(l, "\t CODES:\t", arg1_code, arg2_code)

## Notes
* The anaphora tags are denoted as 'T' tags
* They occur in the E (Event) tags under a causal relation
* They always seem to be accompanied by both a causal relation, and an 'R' (relation) tag which denotes the antecedent relationship

## Recommended TODOs
* Use the antecedent relation to look up the arg2 tag, and use that to resolve the concept code
* Add new concept codes and crels to denote the resolved anaphoric refs
* Resolve these as ("Anaphor-" + code) to differentiate them from the regular tags, so we don't pull them in as normal tags accidentally when re-running previous code.
* i.e. treat these as entirely new tags.

## TODO
 * Are all anaphora tags part of a causal relation? My earlier analysis seemed to contradict this.
 * Study output of the SC Bi-LSTM 'fixed' concept code tagger by computing error metrics over them